In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.utils import to_categorical

train_df = pd.read_csv(r"C:\Users\BIG3-05\Downloads\open\train.csv")
test_df = pd.read_csv(r"C:\Users\BIG3-05\Downloads\open\test.csv")

# LabelEncoder를 사용하여 라벨을 숫자로 변환
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df['label'])
num_classes = len(label_encoder.classes_)
# 라벨을 one-hot 인코딩
train_labels_onehot = to_categorical(train_labels_encoded, num_classes)

# U-Net 모델 아키텍처 정의
def unet(input_shape, num_classes):
    inputs = Input(input_shape)

    # Contracting Path
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expansive Path
    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    # Output layer with the number of classes channels
    conv10 = Conv2D(3, 1, activation='softmax')(conv9)

    # Flatten the output to get only the label values
    output = layers.Flatten()(conv10)

    model = Model(inputs=[inputs], outputs=[conv10])

    return model

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 이미지 스케일링 및 데이터 분할

# Train 데이터 제너레이터 생성
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="upscale_img_path",             # 이미지 파일 경로 컬럼
    y_col="label",                # 라벨 컬럼
    target_size=(256, 256),       # 고해상도 이미지 크기
    batch_size=32,
    class_mode="input",
    subset='training'             # 'training' subset 선택 (훈련용 데이터)
)

# U-Net 모델 생성 및 컴파일
input_shape = (256, 256, 3)  # 입력 이미지 크기
unet_model = unet(input_shape, num_classes)
unet_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# 모델 훈련
unet_model.fit(train_generator, epochs=10)


Found 12668 validated image filenames.
Epoch 1/10


C:\Users\BIG3-05\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 21/396 ━━━━━━━━━━━━━━━━━━━━ 2:18:36 22s/step - accuracy: 0.3172 - loss: 363205184454656.0000

KeyboardInterrupt: 

In [ ]:
train_df

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher
...,...,...,...
15829,./train/TRAIN_15829.jpg,./upscale_train/TRAIN_15829.png,Common Myna
15830,./train/TRAIN_15830.jpg,./upscale_train/TRAIN_15830.png,Common Kingfisher
15831,./train/TRAIN_15831.jpg,./upscale_train/TRAIN_15831.png,Cattle Egret
15832,./train/TRAIN_15832.jpg,./upscale_train/TRAIN_15832.png,Ruddy Shelduck


In [ ]:


# 테스트 이미지 전처리를 위해 동일한 ImageDataGenerator 사용
test_datagen = ImageDataGenerator(rescale=1./255)

# 데이터프레임에서 테스트 이미지 흐름 생성
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="img_path",
    y_col=None,
    target_size=(64, 64),
    batch_size=32,
    class_mode=None,  # 테스트 데이터에는 라벨이 없습니다
    shuffle=False  # 순서를 유지하기 위해 셔플하지 않습니다
)

# 테스트 이미지로 예측
predictions = model.predict(test_generator)

# 예측 결과 디코딩
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))
test_df['predicted_label'] = predicted_labels

# 'img_path' 열을 제외하고 CSV 파일로 저장
test_df.drop(columns=['img_path'], inplace=True)
test_df.to_csv("test_predicted.csv", index=False)
